In [46]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [47]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [48]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [49]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [50]:
# data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
# print(data_pk2_path)
print(media_pk2_path)

/Users/hodung/Desktop/Workspace/silkroad/pk2-extractor/LoadItemData/data/Media.pk2


In [51]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [52]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [53]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [54]:
_media_file_stream = open(media_pk2_path, 'rb')

In [55]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [56]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [57]:
blowfish_checksum_decoded = "Joymax Pak File"

In [58]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [59]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [60]:
Root = read_blocks_at(256)

In [61]:
blocks_in_memory = {
  "": Root
}

In [62]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [63]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [64]:
text_data_directory = os.path.join('server_dep', 'silkroad', 'textdata')
text_data_directory

'server_dep/silkroad/textdata'

## Load Text Data

### ItemData.txt

In [65]:
text_item_data_list_file = os.path.join(text_data_directory, 'ItemData.txt')
text_item_data_list_file

'server_dep/silkroad/textdata/ItemData.txt'

In [66]:
text_item_data_files = get_lines(text_item_data_list_file)
text_item_data_files

['ItemData_100.txt',
 'ItemData_1000.txt',
 'ItemData_10150.txt',
 'ItemData_10300.txt',
 'ItemData_10350.txt',
 'ItemData_10400.txt',
 'ItemData_10450.txt',
 'ItemData_1050.txt',
 'ItemData_10500.txt',
 'ItemData_10550.txt',
 'ItemData_10600.txt',
 'ItemData_10650.txt',
 'ItemData_10700.txt',
 'ItemData_10750.txt',
 'ItemData_10800.txt',
 'ItemData_10850.txt',
 'ItemData_10900.txt',
 'ItemData_10950.txt',
 'ItemData_1100.txt',
 'ItemData_11000.txt',
 'ItemData_11050.txt',
 'ItemData_11100.txt',
 'ItemData_11150.txt',
 'ItemData_11200.txt',
 'ItemData_11250.txt',
 'ItemData_11300.txt',
 'ItemData_11350.txt',
 'ItemData_11400.txt',
 'ItemData_11450.txt',
 'ItemData_1150.txt',
 'ItemData_11500.txt',
 'ItemData_11550.txt',
 'ItemData_11600.txt',
 'ItemData_11650.txt',
 'ItemData_11700.txt',
 'ItemData_11750.txt',
 'ItemData_11800.txt',
 'ItemData_11850.txt',
 'ItemData_11900.txt',
 'ItemData_11950.txt',
 'ItemData_1200.txt',
 'ItemData_12000.txt',
 'ItemData_12050.txt',
 'ItemData_12100.t

In [67]:
df_text_item_data = pd.DataFrame(columns=range(160))

for text_item_data_file in text_item_data_files:
  file_name = os.path.join(text_data_directory, text_item_data_file)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 160:
      continue
    df_text_item_data.loc[len(df_text_item_data)] = values
    


In [68]:
df_text_item_data

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
0,1,55,ITEM_ETC_CURE_ALL_01,?? ??(?),xxx,SN_ITEM_ETC_CURE_ALL_01,SN_ITEM_ETC_CURE_ALL_01_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,-1,xxx,0,0
1,1,56,ITEM_ETC_CURE_ALL_02,?? ??(?),xxx,SN_ITEM_ETC_CURE_ALL_02,SN_ITEM_ETC_CURE_ALL_02_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,-1,xxx,0,0
2,1,57,ITEM_ETC_CURE_ALL_03,?? ??(?),xxx,SN_ITEM_ETC_CURE_ALL_03,SN_ITEM_ETC_CURE_ALL_03_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,-1,xxx,0,0
3,1,58,ITEM_ETC_CURE_ALL_04,?? ?? ?? (?),xxx,SN_ITEM_ETC_CURE_ALL_04,SN_ITEM_ETC_CURE_ALL_04_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,-1,xxx,0,0
4,1,59,ITEM_ETC_CURE_ALL_05,?? ?? ?? (?),xxx,SN_ITEM_ETC_CURE_ALL_05,SN_ITEM_ETC_CURE_ALL_05_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,-1,xxx,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20684,1,945,ITEM_CH_M_CLOTHES_04_LA_B,?? ???? ??,ITEM_CH_W_CLOTHES_04_LA_B,SN_ITEM_CH_CLOTHES_04_LA_B,SN_ITEM_CH_CLOTHES_04_LA_B_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,0,?? ??? ?? ?? ?,10,0
20685,1,946,ITEM_CH_M_CLOTHES_04_LA_C,?? ???? ??,ITEM_CH_W_CLOTHES_04_LA_C,SN_ITEM_CH_CLOTHES_04_LA_C,SN_ITEM_CH_CLOTHES_04_LA_C_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,0,?? ??? ?? ?? ?,10,0
20686,1,947,ITEM_CH_M_CLOTHES_05_LA_A,?? ??? ??,ITEM_CH_W_CLOTHES_05_LA_A,SN_ITEM_CH_CLOTHES_05_LA_A,SN_ITEM_CH_CLOTHES_05_LA_A_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,0,?? ??? ?? ?? ?,10,0
20687,1,948,ITEM_CH_M_CLOTHES_05_LA_B,?? ??? ??,ITEM_CH_W_CLOTHES_05_LA_B,SN_ITEM_CH_CLOTHES_05_LA_B,SN_ITEM_CH_CLOTHES_05_LA_B_TT_DESC,0,0,3,...,-1,xxx,-1,xxx,-1,xxx,0,?? ??? ?? ?? ?,10,0
